In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import udf, col, rand, floor,expr
from pyspark.sql.types import StringType, IntegerType
import random

random.seed(40)

spark = SparkSession \
        .builder \
        .master ("local[*]") \
        .appName ("Exercicio Intro") \
        .getOrCreate()

caminho_arquivo = ('/home/luana/PB_LUANA_NOVAIS/Sprint_8/exercicios/geração-massa de dados/nome_aleatorios.txt')

df_nomes = spark.read.csv(caminho_arquivo,
    header=False,       
    inferSchema=True,   
    sep=",")      

df_nomes = df_nomes.toDF("Nomes")

df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [7]:
def gerar_escolaridade():
    return random.choice(["Fundamental", "Médio", "Superior"])

gerar_escolaridade_udf = udf(gerar_escolaridade, StringType())
df_nomes = df_nomes.withColumn("Escolaridade", gerar_escolaridade_udf())

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet| Fundamental|
|    Jamie Russell|    Superior|
|   Edward Kistler|       Médio|
|    Sheila Maurer|       Médio|
| Donald Golightly|       Médio|
|       David Gray|    Superior|
|      Joy Bennett| Fundamental|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|       Médio|
|    Brian Farrell|    Superior|
+-----------------+------------+
only showing top 10 rows



In [8]:
def gerar_pais():
    return random.choice(["Argentina","Bolívia","Brasil","Chile","Colômbia","Equador","Guiana","Paraguai","Peru",
                          "Suriname","Uruguai","Venezuela","Guiana Francesa"])

gerar_pais_udf = udf(gerar_pais,StringType())
df_nomes = df_nomes.withColumn("Pais", gerar_pais_udf())

df_nomes.show(10)


+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet|    Superior|   Guiana|
|    Jamie Russell|    Superior| Suriname|
|   Edward Kistler| Fundamental|   Guiana|
|    Sheila Maurer| Fundamental|  Uruguai|
| Donald Golightly|    Superior|Argentina|
|       David Gray|       Médio|  Uruguai|
|      Joy Bennett|    Superior|Venezuela|
|      Paul Kriese|       Médio| Paraguai|
|Berniece Ornellas| Fundamental|Argentina|
|    Brian Farrell| Fundamental| Paraguai|
+-----------------+------------+---------+
only showing top 10 rows



In [ ]:
df_nomes = df_nomes.withColumn(
    "AnoNascimento", 
    expr("CAST(rand() * (2010 - 1945 + 1) + 1945 AS INT)")
)

df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|    Superior|   Guiana|         1988|
|    Jamie Russell|    Superior| Suriname|         1957|
|   Edward Kistler| Fundamental|   Guiana|         1998|
|    Sheila Maurer| Fundamental|  Uruguai|         2005|
| Donald Golightly|    Superior|Argentina|         1968|
|       David Gray|       Médio|  Uruguai|         2003|
|      Joy Bennett|    Superior|Venezuela|         1971|
|      Paul Kriese|       Médio| Paraguai|         1972|
|Berniece Ornellas| Fundamental|Argentina|         1978|
|    Brian Farrell| Fundamental| Paraguai|         1972|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [10]:
df_select = df_nomes.filter((col("AnoNascimento") >= 2001))

df_select.show(10)

+---------------+------------+---------------+-------------+
|          Nomes|Escolaridade|           Pais|AnoNascimento|
+---------------+------------+---------------+-------------+
|  Sheila Maurer| Fundamental|        Uruguai|         2005|
|     David Gray|       Médio|        Uruguai|         2003|
| Kara Mcelwaine|    Superior|       Colômbia|         2002|
|  Tracy Herring|    Superior|        Equador|         2003|
| Herbert Morris|    Superior|        Equador|         2009|
|     Daryl Page|       Médio|      Argentina|         2007|
|Kenneth Rayburn|    Superior|Guiana Francesa|         2003|
| Mary Dillahunt|    Superior|       Suriname|         2007|
|  George Miller| Fundamental|         Guiana|         2005|
|Charles Randall| Fundamental|       Suriname|         2006|
+---------------+------------+---------------+-------------+
only showing top 10 rows



In [11]:
df_nomes.createOrReplaceTempView("tabela_nomes")

df_nomes_sec = spark.sql("""
    SELECT Nomes, AnoNascimento
    FROM tabela_nomes
    WHERE AnoNascimento BETWEEN 2001 AND 2010
""")

df_nomes_sec.show(10)


+---------------+-------------+
|          Nomes|AnoNascimento|
+---------------+-------------+
|  Sheila Maurer|         2005|
|     David Gray|         2003|
| Kara Mcelwaine|         2002|
|  Tracy Herring|         2003|
| Herbert Morris|         2009|
|     Daryl Page|         2007|
|Kenneth Rayburn|         2003|
| Mary Dillahunt|         2007|
|  George Miller|         2005|
|Charles Randall|         2006|
+---------------+-------------+
only showing top 10 rows



In [12]:
df_milennials = df_nomes.filter((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994))

num_milennials = df_milennials.count()
print(f"Total de millennials: {num_milennials}")

Total de millennials: 2269391


In [15]:
df_nomes.createOrReplaceTempView("tabela_nomes")

df_nomes_milennials = spark.sql("""
    SELECT COUNT(AnoNascimento) AS TotalMilennials
    FROM tabela_nomes
    WHERE AnoNascimento BETWEEN 1980 AND 1994
""")

df_nomes_milennials.show()

+---------------+
|TotalMilennials|
+---------------+
|        2269391|
+---------------+



In [16]:
df_nomes.createOrReplaceTempView("tabela_nomes")

df_resultado = spark.sql("""
select 
    pais,
    case 
        WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
        WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
        WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
        WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
        ELSE 'Outros'
    END AS Geracao,
        COUNT(*) AS Quantidade
    FROM tabela_nomes
    GROUP BY Pais, Geracao
    ORDER BY Pais ASC, Geracao ASC, Quantidade ASC
""")

df_resultado.show()


+---------+------------+----------+
|     pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    232738|
|Argentina|   Geração X|    174354|
|Argentina|   Geração Z|    186703|
|Argentina| Millennials|    174871|
|  Bolívia|Baby Boomers|    233207|
|  Bolívia|   Geração X|    174625|
|  Bolívia|   Geração Z|    186724|
|  Bolívia| Millennials|    174939|
|   Brasil|Baby Boomers|    234417|
|   Brasil|   Geração X|    174879|
|   Brasil|   Geração Z|    186925|
|   Brasil| Millennials|    174325|
|    Chile|Baby Boomers|    233721|
|    Chile|   Geração X|    175477|
|    Chile|   Geração Z|    185924|
|    Chile| Millennials|    173714|
| Colômbia|Baby Boomers|    233241|
| Colômbia|   Geração X|    174728|
| Colômbia|   Geração Z|    186482|
| Colômbia| Millennials|    174680|
+---------+------------+----------+
only showing top 20 rows

